In [2]:
import re

# Match parameters in URL paths, eg. '{param}', and '{param:int}'
PARAM_REGEX = re.compile("{([a-zA-Z_][a-zA-Z0-9_]*)(:[a-zA-Z_][a-zA-Z0-9_]*)?}")


In [3]:
path = "{int}"

for match in PARAM_REGEX.finditer(path):
    first, second = match.groups("str")
    print(f"{first=}, {second=}")

first='int', second='str'


In [4]:
path = "123"

PARAM_REGEX.match(path)

In [5]:
import typing

def compile_path(
    path: str,
) -> typing.Tuple[typing.Pattern[str], str, typing.Dict[str, Convertor[typing.Any]]]:
    """
    Given a path string, like: "/{username:str}",
    or a host string, like: "{subdomain}.mydomain.org", return a three-tuple
    of (regex, format, {param_name:convertor}).

    regex:      "/(?P<username>[^/]+)"
    format:     "/{username}"
    convertors: {"username": StringConvertor()}
    """
    is_host = not path.startswith("/")

    path_regex = "^"
    path_format = ""
    duplicated_params = set()

    idx = 0
    param_convertors = {}
    for match in PARAM_REGEX.finditer(path):
        param_name, convertor_type = match.groups("str")
        convertor_type = convertor_type.lstrip(":")
        assert (
            convertor_type in CONVERTOR_TYPES
        ), f"Unknown path convertor '{convertor_type}'"
        convertor = CONVERTOR_TYPES[convertor_type]

        path_regex += re.escape(path[idx : match.start()])
        path_regex += f"(?P<{param_name}>{convertor.regex})"

        path_format += path[idx : match.start()]
        path_format += "{%s}" % param_name

        if param_name in param_convertors:
            duplicated_params.add(param_name)

        param_convertors[param_name] = convertor

        idx = match.end()

    if duplicated_params:
        names = ", ".join(sorted(duplicated_params))
        ending = "s" if len(duplicated_params) > 1 else ""
        raise ValueError(f"Duplicated param name{ending} {names} at path {path}")

    if is_host:
        # Align with `Host.matches()` behavior, which ignores port.
        hostname = path[idx:].split(":")[0]
        path_regex += re.escape(hostname) + "$"
    else:
        path_regex += re.escape(path[idx:]) + "$"

    path_format += path[idx:]

    return re.compile(path_regex), path_format, param_convertors

NameError: name 'Convertor' is not defined

In [ ]:
path_regex, path_format, param_convertors = compile_path("https://httpie.org/post/{post_id}/values/{post_id}")

path = "https://httpie.org/post/2/values/1"

match = path_regex.match(path)
if match:
    matched_params = match.groupdict()
    for key, value in matched_params.items():
        print(key, value)

ValueError: Duplicated param name post_id at path https://httpie.org/post/{post_id}/values/{post_id}

In [ ]:
param_convertors

In [ ]:
import urllib.parse

part1 = "https://httpie.org/post"

part2 = "/values/1"

urllib.parse.urljoin(part1, part2)

In [ ]:
urllib.parse.urlsplit("https://httpie.org/post").path

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional, NamedTuple
from arrest import params

class RequestBody(BaseModel):
    id: int = Field(...)
    name: str
    payment_id: Optional[str] = None
    req_id: str = params.Query(default="xyz")

req = RequestBody(id=1, name="abc", payment_id="1234")
req.model_fields
getattr(req, "req_id")

'xyz'

In [ ]:
from arrest.resource import ResourceHandler
from arrest.http import Methods

In [ ]:
handlers = [(Methods.GET, "/"), (Methods.POST, "/payments")]

In [ ]:
ResourceHandler(method=Methods.POST, route="/", request=None, response=None, callback=False)

ValidationError: 1 validation error for ResourceHandler
callback
  Input should be callable [type=callable_type, input_value=False, input_type=bool]
    For further information visit https://errors.pydantic.dev/2.0.3/v/callable_type

In [ ]:
print(1 > 2 and "huzzah" or None)

None


In [ ]:
x = [1, 2]

a, b, *rest = x[0], x[1], x[2:]

In [ ]:
rest[1]

IndexError: list index out of range

In [ ]:
"/a/b/c".strip("/").split("/")

['a', 'b', 'c']

In [ ]:
"/".join(["b"])

'b'

In [ ]:
import re

pat = re.compile("/(?P<username>[^/]+)")

In [ ]:
def replace_params(
    path: str,
    path_params: dict[str, int],
) -> tuple[str, dict[str, str]]:
    for key, value in list(path_params.items()):
        if "{" + key + "}" in path:
            print(key)
            value = str(value)
            path = path.replace("{" + key + "}", value)
            path_params.pop(key)
    return path, path_params

In [ ]:
replace_params("/path/{username:int}", {"username": 123})

('/path/{username:int}', {'username': 123})

In [ ]:
from arrest.converters import compile_path

In [ ]:
pattern, path_format, path_param_types = compile_path("https://example.com/rest/path/{username:uuid}/posts/{post_id:uuid}")

In [ ]:
requested_url = "https://example.com/rest/path"
requested_url in path_format

False

In [ ]:
from pydantic import BaseModel

class MyClass(BaseModel):
    a: int
    b: float


MyClass.validate({"a": 1, "b": "1234"})

/tmp/ipykernel_1240/4065202313.py:8: PydanticDeprecatedSince20: The `validate` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.0.3/migration/
  MyClass.validate({"a": 1, "b": "1234"})


MyClass(a=1, b=1234.0)

In [1]:
from arrest.converters import replace_params
from arrest import converters

replace_params("/{param}", path_params={"param": "xyz"}, param_types={"param": converters.IntegerConverter()})

ValueError: invalid literal for int() with base 10: 'xyz'